In [ ]:
from kafka import KafkaProducer
import pandas as pd
import os
import kagglehub
import time
from datetime import datetime, timedelta

# הגדרת Kafka Producer ששולח מחרוזות רגילות
producer = KafkaProducer(
    bootstrap_servers='localhost:9092',
    value_serializer=lambda v: v.encode('utf-8')
)

def send_to_kafka(df):
    for _, row in df.iterrows():
        row_str = ','.join(map(str, row.values)) 
        producer.send('tennis_daily_data', row_str)

def update_data():
    path = kagglehub.dataset_download("dissfya/atp-tennis-2000-2023daily-pull")
    for filename in os.listdir(path):
        if filename.endswith(".csv"):
            csv_file_path = os.path.join(path, filename)
    df = pd.read_csv(csv_file_path)
    send_to_kafka(df)
    print("Updated data and sent to Kafka at", time.strftime("%H:%M:%S"))

def get_seconds_until_8am():
    """ מחזיר כמה שניות נותרו עד 08:00 בבוקר הבא """
    now = datetime.now()
    target_time = datetime.combine(now.date(), datetime.min.time()) + timedelta(hours=80,minutes=0)
    
    # אם כבר עברנו את 08:00 היום, נקבע את היעד למחר
    if now > target_time:
        target_time += timedelta(days=1)

    return (target_time - now).total_seconds()

# הפעלה ראשונית - שליחת נתונים מיד
update_data()

# חישוב הזמן עד 08:00 והמתנה ישירה
while True:
    seconds_until_8am = get_seconds_until_8am()
    print(f"Sleeping for {seconds_until_8am:.2f} seconds until 08:00 AM...")
    time.sleep(seconds_until_8am)  # המתנה ישירה עד 08:00
    update_data()

Updated data and sent to Kafka at 20:54:28
Sleeping for 31.76 seconds until 08:00 AM...
Updated data and sent to Kafka at 20:55:13
Sleeping for 86386.86 seconds until 08:00 AM...
